# Prompt Preview (Local)
Preview the exact prompts used for SRL/base benchmarking by sampling one problem from a benchmark dataset.

## Usage
- Run this notebook from the repo root (no Colab clone steps).
- Ensure dependencies are installed (`pip install -e .`).
- Choose a benchmark and model type below and run the cells.

In [ ]:
# Ensure repo root is on sys.path for local runs
import sys
from pathlib import Path
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT / 'pyproject.toml').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print('Using project root:', ROOT)


## Configure benchmark and model type

In [ ]:
# Benchmark: amc23, aime24, or aime25
BENCHMARK = "aime25"
MODEL_TYPE = "base"  # "srl" (with <think>) or "base" (no <think>)


## Load one example and build the prompt

In [ ]:
import re
from pathlib import Path
from benchmarks.data_loader import load_benchmark_data

# Load data (first example only)
data = load_benchmark_data(BENCHMARK)
example = data[0]
print(f"Loaded {len(data)} problems; showing first one.\n")

# Extract templates directly from evaluator source (avoids importing vllm)
text = (ROOT / "benchmarks" / "evaluator.py").read_text()
base_match = re.search(r'BASE_PROMPT_TEMPLATE = """(.*?)"""', text, re.S)
srl_match = re.search(r'SRL_PROMPT_TEMPLATE = """(.*?)"""', text, re.S)
base_template = base_match.group(1) if base_match else None
srl_template = srl_match.group(1) if srl_match else None

if MODEL_TYPE == "srl":
    from src.shared.prompts import build_srl_prompt
    prompt = build_srl_prompt(example["problem"], [], include_closing_tag=False)
    print("SRL prompt template from evaluator:\n")
    print(srl_template)
else:
    # Avoid Python .format interpreting the \boxed{} braces; do a simple replacement
    prompt = base_template.replace("{problem}", example["problem"])
    print("Base prompt template from evaluator:\n")
    print(base_template)

print("\n---\nFull prompt with sample problem:\n")
print(prompt)
